In [3]:
import numpy as np
import os
import pandas as pd
from soma import aims, aimsalgo
import matplotlib.pyplot as plt

In [4]:
directory = '/volatile/jl277509/data/UkBioBank/crops/2mm/CINGULATE/mask/'
side = 'R'
crops_dirs = directory+side+'crops/'
skel_subjects = pd.read_csv(directory+side+'skeleton_subject.csv')

In [5]:
vol_list = []
fm = aims.FastMarching()
for i, subject in enumerate(skel_subjects.Subject):
    vol = aims.read(crops_dirs+subject+'_cropped_skeleton.nii.gz')
    outside = 0
    other_outside = 11
    bottom_val = 30
    ss_val = 60

    # change all other values to ss_val
    vol_tmp = aims.Volume(vol)
    vol[vol.np != ss_val] = ss_val
    vol[vol_tmp.np == bottom_val] = bottom_val
    vol[vol_tmp.np == outside] = outside
    vol[vol_tmp.np == other_outside] = outside

    vol_list.append(vol.np)

    # propagation dans ss_val, avec outside non-atteignable
    aimsalgo.AimsDistanceFrontPropagation(vol, ss_val, outside, 3, 3, 3, 50, False)
    #aims.write(vol, directory+side+f'distbottom/{subject}_cropped_distbottom.nii.gz')

In [6]:
arr_distbottom_all = np.stack(vol_list)
arr_distbottom_all.shape

(21052, 16, 37, 37, 1)

In [ ]:
np.save(directory+side+'distbottom.npy', arr_distbottom_all)

In [14]:
np.unique(vol)

array([    0,   100,   141,   173,   200,   241,   273,   282,   300,
         314,   341,   373,   382,   400,   414,   423,   441,   446,
         455,   473,   482,   514,   523,   555,   582,   587, 32500,
       32501], dtype=int16)

In [6]:
for subject in skel_subjects.Subject:
    vol1 = aims.read(crops_dirs+subject+'_cropped_skeleton.nii.gz')
    vol2 = aims.read(directory+side+f'distbottom/{subject}_cropped_distbottom.nii.gz')

In [10]:
# check bottoms
print(np.sum(vol1.np==30))
print(np.sum(vol2.np==0))

87
87


In [12]:
# check that all voxels are caught
print(np.sum(vol1.np>0))
print(np.sum(vol2.np<10000))

376
374
